In [1]:
import pandas as pd

from SMILESX import main, inference
%matplotlib inline

Using TensorFlow backend.


#### Definition of data

In [2]:
validation_data_dir = "./validation_data/"

In [3]:
extension = '.csv'

In [4]:
data_name = 'FreeSolv' # FreeSolv, ESOL, Lipophilicity
prop_tag = ''

In [5]:
if data_name == 'FreeSolv':
    data_filename = 'FreeSolv_SAMPL'
    prop_tag = 'expt'
elif data_name == 'ESOL':
    data_filename = 'ESOL_delaney-processed'
    prop_tag = 'measured log solubility in mols per litre'
elif data_name == 'Lipophilicity':
    data_filename = 'Lipophilicity'
    prop_tag = 'exp'
else:
    data_filename = data_name
    prop_tag = prop_tag

In [6]:
sol_data = pd.read_csv(validation_data_dir+data_filename+extension)

In [7]:
sol_data.head(3)

,Unnamed: 0,iupac,smiles,expt,calc
0,0,"4-methoxy-N,N-dimethyl-benzamide",COc1ccc(C(=O)N(C)C)cc1,-11.01,-9.625
1,1,methanesulfonyl chloride,CS(=O)(=O)Cl,-4.87,-6.219
2,2,3-methylbut-1-ene,C=CC(C)C,1.83,2.452


In [8]:
sol_data = sol_data[['smiles',prop_tag]]

In [9]:
sol_data.head()

,smiles,expt
0,COc1ccc(C(=O)N(C)C)cc1,-11.01
1,CS(=O)(=O)Cl,-4.87
2,C=CC(C)C,1.83
3,CCc1cnccn1,-5.45
4,CCCCCCCO,-4.21


In [10]:
sol_data.shape

(642, 2)

#### Hyperparameters optimization with GPyOpt (Bayesian optimization)

In [11]:
dhyp_range = [int(2**itn) for itn in range(3,11)] # 
dalpha_range = [float(ialpha/10.) for ialpha in range(20,40,1)] # Adam's learning rate = 10^(-dalpha_range)

if data_name != 'Lipophilicity':
    bounds = [
        {'name': 'lstmunits', 'type': 'discrete', 'domain': dhyp_range}, 
        {'name': 'denseunits', 'type': 'discrete', 'domain': dhyp_range}, 
        {'name': 'embedding', 'type': 'discrete', 'domain': dhyp_range}, 
        {'name': 'batchsize', 'type': 'discrete', 'domain': dhyp_range}, 
        {'name': 'lrate', 'type': 'discrete', 'domain': dalpha_range}
    ]
else:
    bounds = [
        {'name': 'lstmunits', 'type': 'discrete', 'domain': dhyp_range}, 
        {'name': 'denseunits', 'type': 'discrete', 'domain': dhyp_range}, 
        {'name': 'embedding', 'type': 'discrete', 'domain': dhyp_range}, 
        {'name': 'batchsize', 'type': 'discrete', 'domain': (1024, 1024)}, # fixed
        {'name': 'lrate', 'type': 'discrete', 'domain': (3, 3)} # fixed
    ]

In [12]:
main.Main(data=sol_data, 
          data_name=data_name, 
          data_units='', 
          bayopt_bounds=bounds, 
          k_fold_number = 3, 
          augmentation = True, 
          outdir = "./data/", 
          bayopt_n_epochs = 1,
          bayopt_n_rounds = 1, 
          bayopt_it_factor = 1, 
          bayopt_on = True, 
          lstmunits_ref = 128, 
          denseunits_ref = 16, 
          embedding_ref = 32, 
          n_gpus = 1, 
          bridge_type = 'NVLink', 
          batch_size_ref = 8,
          alpha_ref = 22, 
          patience = 50, 
          n_epochs = 100)

***SMILES_X starts...***


******
***Fold #0 initiated...***
******
***Sampling and splitting of the dataset.***

Scaler: RobustScaler(copy=True, quantile_range=(5.0, 95.0), with_centering=True,
             with_scaling=True)
Train/valid/test splits: 0.80/0.10/0.10


***Data augmentation to True***

Enumerated SMILES:
	Training set: 4285
	Validation set: 501
	Test set: 569

***Tokenization of SMILES.***

Examples of tokenized SMILES from a training set:
[[' ', 'C', 'Cl', ' '], [' ', 'Cl', 'C', ' '], [' ', 'C', 'C', 'c', '1', 'c', 'c', 'n', 'c', 'c', '1', ' '], [' ', 'C', '(', 'c', '1', 'c', 'c', 'n', 'c', 'c', '1', ')', 'C', ' '], [' ', 'c', '1', '(', 'C', 'C', ')', 'c', 'c', 'n', 'c', 'c', '1', ' ']]

Number of tokens only present in a training set: 32

Number of tokens only present in a validation set: 27
Is the validation set a subset of the training set: True
What are the tokens by which they differ: set()

Number of tokens only present in a test set: 27
Is the test set a subset o

Epoch 21/100
134/134 [==============================] - 1s 8ms/step - loss: 0.0092 - mean_absolute_error: 0.0732 - mean_squared_error: 0.0092 - val_loss: 0.0097 - val_mean_absolute_error: 0.0734 - val_mean_squared_error: 0.0097
Epoch 22/100
134/134 [==============================] - 1s 8ms/step - loss: 0.0089 - mean_absolute_error: 0.0724 - mean_squared_error: 0.0089 - val_loss: 0.0086 - val_mean_absolute_error: 0.0711 - val_mean_squared_error: 0.0086
Epoch 23/100
134/134 [==============================] - 1s 6ms/step - loss: 0.0088 - mean_absolute_error: 0.0717 - mean_squared_error: 0.0088 - val_loss: 0.0127 - val_mean_absolute_error: 0.0885 - val_mean_squared_error: 0.0127
Epoch 24/100
134/134 [==============================] - 1s 9ms/step - loss: 0.0084 - mean_absolute_error: 0.0691 - mean_squared_error: 0.0084 - val_loss: 0.0119 - val_mean_absolute_error: 0.0862 - val_mean_squared_error: 0.0119
Epoch 25/100
134/134 [==============================] - 1s 10ms/step - loss: 0.0083 - me

Epoch 57/100
134/134 [==============================] - 1s 7ms/step - loss: 0.0051 - mean_absolute_error: 0.0535 - mean_squared_error: 0.0051 - val_loss: 0.0084 - val_mean_absolute_error: 0.0675 - val_mean_squared_error: 0.0084
Epoch 58/100
134/134 [==============================] - 1s 7ms/step - loss: 0.0051 - mean_absolute_error: 0.0538 - mean_squared_error: 0.0051 - val_loss: 0.0119 - val_mean_absolute_error: 0.0723 - val_mean_squared_error: 0.0119
Epoch 59/100
134/134 [==============================] - 1s 8ms/step - loss: 0.0050 - mean_absolute_error: 0.0528 - mean_squared_error: 0.0050 - val_loss: 0.0072 - val_mean_absolute_error: 0.0644 - val_mean_squared_error: 0.0072
Epoch 60/100
134/134 [==============================] - 1s 6ms/step - loss: 0.0054 - mean_absolute_error: 0.0546 - mean_squared_error: 0.0054 - val_loss: 0.0086 - val_mean_absolute_error: 0.0646 - val_mean_squared_error: 0.0086
Epoch 61/100
134/134 [==============================] - 1s 8ms/step - loss: 0.0048 - mea

Epoch 93/100
134/134 [==============================] - 1s 6ms/step - loss: 0.0037 - mean_absolute_error: 0.0440 - mean_squared_error: 0.0037 - val_loss: 0.0068 - val_mean_absolute_error: 0.0601 - val_mean_squared_error: 0.0068
Epoch 94/100
134/134 [==============================] - 1s 6ms/step - loss: 0.0038 - mean_absolute_error: 0.0449 - mean_squared_error: 0.0038 - val_loss: 0.0074 - val_mean_absolute_error: 0.0617 - val_mean_squared_error: 0.0074
Epoch 95/100
134/134 [==============================] - 1s 7ms/step - loss: 0.0036 - mean_absolute_error: 0.0443 - mean_squared_error: 0.0036 - val_loss: 0.0077 - val_mean_absolute_error: 0.0638 - val_mean_squared_error: 0.0077
Epoch 96/100
134/134 [==============================] - 1s 6ms/step - loss: 0.0036 - mean_absolute_error: 0.0441 - mean_squared_error: 0.0036 - val_loss: 0.0098 - val_mean_absolute_error: 0.0686 - val_mean_squared_error: 0.0098
Epoch 97/100
134/134 [==============================] - 1s 6ms/step - loss: 0.0034 - mea

67/67 [==============================] - 0s 6ms/step - loss: 0.0261 - mean_absolute_error: 0.1280 - mean_squared_error: 0.0261 - val_loss: 0.0271 - val_mean_absolute_error: 0.1348 - val_mean_squared_error: 0.0271
Epoch 13/100
67/67 [==============================] - 1s 8ms/step - loss: 0.0275 - mean_absolute_error: 0.1321 - mean_squared_error: 0.0275 - val_loss: 0.0306 - val_mean_absolute_error: 0.1420 - val_mean_squared_error: 0.0306
Epoch 14/100
67/67 [==============================] - 0s 6ms/step - loss: 0.0249 - mean_absolute_error: 0.1270 - mean_squared_error: 0.0249 - val_loss: 0.0297 - val_mean_absolute_error: 0.1417 - val_mean_squared_error: 0.0297
Epoch 15/100
67/67 [==============================] - 0s 6ms/step - loss: 0.0242 - mean_absolute_error: 0.1256 - mean_squared_error: 0.0242 - val_loss: 0.0273 - val_mean_absolute_error: 0.1354 - val_mean_squared_error: 0.0273
Epoch 16/100
67/67 [==============================] - 0s 7ms/step - loss: 0.0257 - mean_absolute_error: 0.129

67/67 [==============================] - 0s 7ms/step - loss: 0.0113 - mean_absolute_error: 0.0839 - mean_squared_error: 0.0113 - val_loss: 0.0218 - val_mean_absolute_error: 0.1085 - val_mean_squared_error: 0.0218
Epoch 49/100
67/67 [==============================] - 0s 7ms/step - loss: 0.0110 - mean_absolute_error: 0.0816 - mean_squared_error: 0.0110 - val_loss: 0.0194 - val_mean_absolute_error: 0.1006 - val_mean_squared_error: 0.0194
Epoch 50/100
67/67 [==============================] - 0s 7ms/step - loss: 0.0101 - mean_absolute_error: 0.0785 - mean_squared_error: 0.0101 - val_loss: 0.0221 - val_mean_absolute_error: 0.1120 - val_mean_squared_error: 0.0221
Epoch 51/100
67/67 [==============================] - 0s 6ms/step - loss: 0.0105 - mean_absolute_error: 0.0782 - mean_squared_error: 0.0105 - val_loss: 0.0202 - val_mean_absolute_error: 0.1011 - val_mean_squared_error: 0.0202
Epoch 52/100
67/67 [==============================] - 0s 7ms/step - loss: 0.0106 - mean_absolute_error: 0.080

67/67 [==============================] - 0s 6ms/step - loss: 0.0063 - mean_absolute_error: 0.0596 - mean_squared_error: 0.0063 - val_loss: 0.0174 - val_mean_absolute_error: 0.0896 - val_mean_squared_error: 0.0174
Epoch 85/100
67/67 [==============================] - 0s 6ms/step - loss: 0.0064 - mean_absolute_error: 0.0603 - mean_squared_error: 0.0064 - val_loss: 0.0177 - val_mean_absolute_error: 0.0882 - val_mean_squared_error: 0.0177
Epoch 86/100
67/67 [==============================] - 0s 6ms/step - loss: 0.0061 - mean_absolute_error: 0.0576 - mean_squared_error: 0.0061 - val_loss: 0.0185 - val_mean_absolute_error: 0.0978 - val_mean_squared_error: 0.0185
Epoch 87/100
67/67 [==============================] - 0s 6ms/step - loss: 0.0056 - mean_absolute_error: 0.0562 - mean_squared_error: 0.0056 - val_loss: 0.0163 - val_mean_absolute_error: 0.0829 - val_mean_squared_error: 0.0163
Epoch 88/100
67/67 [==============================] - 0s 6ms/step - loss: 0.0059 - mean_absolute_error: 0.057

273/273 [==============================] - 1s 5ms/step - loss: 0.0229 - mean_absolute_error: 0.1194 - mean_squared_error: 0.0229 - val_loss: 0.0191 - val_mean_absolute_error: 0.1096 - val_mean_squared_error: 0.0191
Epoch 6/100
273/273 [==============================] - 1s 5ms/step - loss: 0.0202 - mean_absolute_error: 0.1131 - mean_squared_error: 0.0202 - val_loss: 0.0180 - val_mean_absolute_error: 0.1061 - val_mean_squared_error: 0.0180
Epoch 7/100
273/273 [==============================] - 1s 5ms/step - loss: 0.0186 - mean_absolute_error: 0.1079 - mean_squared_error: 0.0186 - val_loss: 0.0167 - val_mean_absolute_error: 0.1005 - val_mean_squared_error: 0.0167
Epoch 8/100
273/273 [==============================] - 1s 5ms/step - loss: 0.0176 - mean_absolute_error: 0.1052 - mean_squared_error: 0.0176 - val_loss: 0.0183 - val_mean_absolute_error: 0.1079 - val_mean_squared_error: 0.0183
Epoch 9/100
273/273 [==============================] - 1s 5ms/step - loss: 0.0182 - mean_absolute_error:

273/273 [==============================] - 1s 5ms/step - loss: 0.0036 - mean_absolute_error: 0.0446 - mean_squared_error: 0.0036 - val_loss: 0.0127 - val_mean_absolute_error: 0.0852 - val_mean_squared_error: 0.0127
Epoch 42/100
273/273 [==============================] - 1s 5ms/step - loss: 0.0033 - mean_absolute_error: 0.0435 - mean_squared_error: 0.0033 - val_loss: 0.0100 - val_mean_absolute_error: 0.0655 - val_mean_squared_error: 0.0100
Epoch 43/100
273/273 [==============================] - 1s 5ms/step - loss: 0.0033 - mean_absolute_error: 0.0435 - mean_squared_error: 0.0033 - val_loss: 0.0102 - val_mean_absolute_error: 0.0632 - val_mean_squared_error: 0.0102
Epoch 44/100
273/273 [==============================] - 1s 5ms/step - loss: 0.0042 - mean_absolute_error: 0.0472 - mean_squared_error: 0.0042 - val_loss: 0.0104 - val_mean_absolute_error: 0.0687 - val_mean_squared_error: 0.0104
Epoch 45/100
273/273 [==============================] - 2s 6ms/step - loss: 0.0028 - mean_absolute_er

Epoch 77/100
273/273 [==============================] - 1s 5ms/step - loss: 0.0019 - mean_absolute_error: 0.0322 - mean_squared_error: 0.0019 - val_loss: 0.0121 - val_mean_absolute_error: 0.0730 - val_mean_squared_error: 0.0121
Epoch 78/100
273/273 [==============================] - 2s 6ms/step - loss: 0.0013 - mean_absolute_error: 0.0284 - mean_squared_error: 0.0013 - val_loss: 0.0116 - val_mean_absolute_error: 0.0654 - val_mean_squared_error: 0.0116
Epoch 79/100
273/273 [==============================] - 2s 6ms/step - loss: 0.0019 - mean_absolute_error: 0.0329 - mean_squared_error: 0.0019 - val_loss: 0.0121 - val_mean_absolute_error: 0.0693 - val_mean_squared_error: 0.0121
Epoch 80/100
273/273 [==============================] - 2s 6ms/step - loss: 0.0016 - mean_absolute_error: 0.0306 - mean_squared_error: 0.0016 - val_loss: 0.0105 - val_mean_absolute_error: 0.0614 - val_mean_squared_error: 0.0105
Epoch 81/100
273/273 [==============================] - 2s 6ms/step - loss: 0.0012 - mea

In [13]:
pred_from_ens = inference.Inference(data_name=data_name, 
                                    smiles_list = ['CC','CCC','C=O','ABC','DEF'], 
                                    data_units = '',
                                    k_fold_number = 3,
                                    augmentation = True, 
                                    outdir = "./data/")

***SMILES_X for inference starts...***


***Checking the SMILES list for inference***

***Data augmentation.***

Enumerated SMILES: 5

***Tokenization of SMILES.***

Full vocabulary: ['pad', 'unk', '[C@@H]', '\\', '=', '/', '-', 'N', '4', 'S', '#', 'c', ')', 'F', 'C', '1', '3', 's', 'Cl', '[C@H]', 'I', '5', '[N+]', '[nH]', '(', ' ', 'P', 'O', '2', '[C@@]', '[S+2]', 'Br', '[C@]', 'n', '[O-]']
Of size: 35

Maximum length of tokenized SMILES: 51 tokens

***Inference of SMILES property done.***


In [14]:
pred_from_ens

,SMILES,ens_pred_mean,ens_pred_sd
0,CC,0.396276,0.0142014
1,CCC,0.42776,0.0132425
2,C=O,-0.0164781,0.0243222
